In [ ]:
# ===== Step 0: Install and Import Required Libraries =====
# ===== ステップ0: 必要なライブラリのインストールとインポート =====
!pip install emcee corner tqdm

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad
from astropy.coordinates import SkyCoord
from astropy import units as u
import emcee
import corner
from tqdm import tqdm
import os

# 日本語フォント設定
!apt-get -y install fonts-noto-cjk -qq
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

# ===== Step 1: Preparation of SNe Data (No Changes) =====
# ===== ステップ1: SNeデータの準備 (変更なし) =====
sne_data_file = '/content/Pantheon+SH0ES.dat'
sne_cov_file = '/content/Pantheon+SH0ES_STAT+SYS.cov'
pantheon_data = pd.read_csv(sne_data_file, sep='\\s+', comment='#')
N_sne = len(pantheon_data)
cov_matrix_flat = np.loadtxt(sne_cov_file, skiprows=1)
cov_matrix = cov_matrix_flat.reshape((N_sne, N_sne))
print("共分散行列の逆行列を計算中...")
C_inv = np.linalg.inv(cov_matrix)
print("計算完了。")
l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value
sn_coords_icrs = SkyCoord(ra=pantheon_data['RA'], dec=pantheon_data['DEC'], unit='deg', frame='icrs')
sn_coords_galactic = sn_coords_icrs.galactic
n_vectors_sn = sn_coords_galactic.cartesian.xyz.value.T
pantheon_data['cos_theta'] = np.dot(n_vectors_sn, d_vector)


# ===== Step 2: Definition of Final Model and Likelihood Function =====
# ===== ステップ2: 最終モデルと尤度関数の定義 =====
c_kms = 299792.458
f_baryon = 0.156
f_cdm = 1.0 - f_baryon

def p2(ct): return 0.5 * (3 * ct**2 - 1)
def r_iso_transition_model(a, R_early, R_late, a_trans, delta_a):
    return R_late + (R_early - R_late) * 0.5 * (1 - np.tanh((a - a_trans) / delta_a))

def hubble_parameter_interaction(z, cos_theta, H0, Omega_m, A2, R_early, R_late, a_trans, delta_a, Gamma0):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_iso = r_iso_transition_model(a, R_early, R_late, a_trans, delta_a)
    r_lock = np.maximum(0, r_iso * (1 + A2 * p2(cos_theta)))
    interaction_gamma = Gamma0 * (1.0 - r_lock)
    matter_term = (Omega_m / a**3) * (a**(3 * interaction_gamma))
    h2 = (Omega_rad / a**4) + matter_term + Omega_L
    if h2 < 0: return np.inf
    return H0 * np.sqrt(h2)

# ★★★ ここを修正 ★★★
# get_mu_theory_loop は、8つの宇宙論パラメータを直接受け取るように修正
def get_mu_theory_loop(cosmo_params, data):
    # cosmo_paramsは既に8つのパラメータなので、ここでのスライスは不要
    H0, Omega_m, A2, R_early, R_late, a_trans, delta_a, Gamma0 = cosmo_params
    z_obs = data['zCMB'].values
    cos_theta_obs = data['cos_theta'].values
    mu_th_array = np.zeros_like(z_obs)
    for i in range(len(z_obs)):
        integrand = lambda z_prime: 1.0 / hubble_parameter_interaction(z_prime, cos_theta_obs[i], *cosmo_params)
        chi, _ = quad(integrand, 0, z_obs[i])
        DL = c_kms * (1 + z_obs[i]) * chi
        mu_th_array[i] = 5 * np.log10(DL) + 25 if DL > 0 else np.inf
    return mu_th_array

def log_likelihood_sne_with_cov(params, data, C_inv):
    M = params[-1]
    cosmo_params = params[:-1]
    mu_obs = data['m_b_corr'].values
    mu_th_model = get_mu_theory_loop(cosmo_params, data)
    if np.any(np.isinf(mu_th_model)): return -np.inf
    diff = mu_obs - (mu_th_model + M)
    chi2_sne = diff @ C_inv @ diff
    return -0.5 * chi2_sne

def log_prior(params):
    H0, Omega_m, A2, R_early, R_late, a_trans, delta_a, Gamma0, M = params
    if 60.0 < H0 < 80.0 and 0.1 < Omega_m < 0.5 and \
       0.0 < A2 < 1.0 and 0.0 < R_early < 2.0 and \
       0.5 < R_late < 1.5 and 0.1 < a_trans < 0.9 and \
       0.01 < delta_a < 0.5 and 0.0 < Gamma0 < 0.1 and \
       -20.0 < M < -18.0:
        return 0.0
    return -np.inf

def log_probability_sne_only(params, data_sne, C_inv_sne):
    lp = log_prior(params)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_sne_with_cov(params, data_sne, C_inv_sne)


# ===== Step 3: Run MCMC (with Pause and Resume Functionality) =====
# ===== ステップ3: MCMCの実行 (中断・再開機能付き) =====
ndim = 9
nwalkers = 32
data_sne_to_use = pantheon_data
C_inv_to_use = C_inv
total_nsteps = 10000
burn_in = 2000
filename = "sne_interaction_model_backend.h5"

if os.path.exists(filename):
    print("以前の実行ファイルが見つかりました。計算を再開します。")
    backend = emcee.backends.HDFBackend(filename, read_only=False)
    initial_state = backend.get_last_sample()
    n_steps_done = backend.iteration
    print(f"完了済みステップ数: {n_steps_done}")
else:
    print("新しいMCMC実行を開始します。")
    backend = emcee.backends.HDFBackend(filename)
    backend.reset(nwalkers, ndim)
    initial_params = np.array([70.0, 0.3, 0.1, 1.0, 1.0, 0.5, 0.1, 0.01, -19.3])
    initial_state = initial_params + 1e-3 * np.random.randn(nwalkers, ndim)
    n_steps_done = 0

steps_to_run = total_nsteps - n_steps_done
sampler = emcee.EnsembleSampler(nwalkers, ndim, log_probability_sne_only,
                                args=(data_sne_to_use, C_inv_to_use),
                                backend=backend)

if steps_to_run > 0:
    print(f"残り {steps_to_run} ステップのMCMCサンプリングを実行します...")
    sampler.run_mcmc(initial_state, steps_to_run, progress=True)
    print("MCMCサンプリングが完了しました。")
else:
    print("計算は既に完了しています。")


# ===== Step 4: Visualization of Results =====
# ===== ステップ4: 結果の可視化 =====
flat_samples = sampler.get_chain(discard=burn_in, thin=15, flat=True)
labels = [r"$H_0$", r"$\Omega_m$", r"$A_2$", r"$R_{early}$", r"$R_{late}$", r"$a_{trans}$", r"$\Delta a$", r"$\Gamma_0$", r"$M$"]
fig = corner.corner(flat_samples, labels=labels, quantiles=[0.16, 0.5, 0.84], show_titles=True, title_kwargs={"fontsize": 12})
plt.show()

print("\n--- パラメータ推定結果 (SNe単独, 最終モデル, 本番実行) ---")
for i in range(ndim):
    mcmc = np.percentile(flat_samples[:, i], [16, 50, 84])
    q = np.diff(mcmc)
    print(f"{labels[i]} = {mcmc[1]:.4f} +{q[1]:.4f} -{q[0]:.4f}")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from scipy.integrate import quad
from astropy.coordinates import SkyCoord
from astropy import units as u
from tqdm import tqdm

# 日本語フォント設定
!apt-get -y install fonts-noto-cjk -qq
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
font_prop = fm.FontProperties(fname=font_path)
plt.rcParams["font.family"] = font_prop.get_name()
plt.rcParams["axes.unicode_minus"] = False

# ===== Step 1: Model Definition and Preparation of Observational Data =====
# ===== ステップ1: モデル定義と観測データの準備 =====
c_kms = 299792.458
f_baryon = 0.156
f_cdm = 1.0 - f_baryon

def p2(ct): return 0.5 * (3 * ct**2 - 1)
def r_iso_transition_model(a, R_early, R_late, a_trans, delta_a):
    return R_late + (R_early - R_late) * 0.5 * (1 - np.tanh((a - a_trans) / delta_a))

def hubble_parameter_interaction(z, cos_theta, H0, Omega_m, A2, R_early, R_late, a_trans, delta_a, Gamma0):
    a = 1.0 / (1.0 + z)
    Omega_rad = 9.24e-5
    Omega_L = 1.0 - Omega_m - Omega_rad
    r_iso = r_iso_transition_model(a, R_early, R_late, a_trans, delta_a)
    r_lock = np.maximum(0, r_iso * (1 + A2 * p2(cos_theta)))
    interaction_gamma = Gamma0 * (1.0 - r_lock)
    matter_term = (Omega_m / a**3) * (a**(3 * interaction_gamma))
    h2 = (Omega_rad / a**4) + matter_term + Omega_L
    if h2 < 0: return np.inf
    return H0 * np.sqrt(h2)

# --- Loading Observational Data ---
# --- 観測データの読み込み ---
sne_data_file = '/content/Pantheon+SH0ES.dat'
pantheon_data = pd.read_csv(sne_data_file, sep='\\s+', comment='#')
# All Data Points Will Be Used for Plotting
# プロットするデータは全点を使用
pantheon_plot_data = pantheon_data.reset_index(drop=True)


# ===== Step 2: ★Set Final Parameters Obtained from Production Run★ =====
# ===== ステップ2: ★本番実行で得られた最終パラメータを設定★ =====
best_fit_params = {
    "H0": 70.8049,
    "Omega_m": 0.3871,
    "A2": 0.2966,
    "R_early": 0.6074,
    "R_late": 0.8998,
    "a_trans": 0.5853,
    "delta_a": 0.2577,
    "Gamma0": 0.0405,
    "M": -19.2987
}
cosmo_params_tuple = (best_fit_params["H0"], best_fit_params["Omega_m"], best_fit_params["A2"],
                      best_fit_params["R_early"], best_fit_params["R_late"], best_fit_params["a_trans"],
                      best_fit_params["delta_a"], best_fit_params["Gamma0"])
M_fit = best_fit_params["M"]


# ===== Step 3: Calculation of Theoretical Curve =====
# ===== ステップ3: 理論曲線の計算 =====
z_theory = np.linspace(0.01, 2.3, 200)

def get_mu_theory_for_curve(z_array, cos_theta, params):
    mu_th_array = np.zeros_like(z_array)
    for i, z in enumerate(tqdm(z_array, desc=f"Calculating curve for cos(theta)={cos_theta:.1f}")):
        integrand = lambda z_prime: 1.0 / hubble_parameter_interaction(z_prime, cos_theta, *params)
        chi, _ = quad(integrand, 0, z)
        DL = c_kms * (1 + z) * chi
        mu_th_array[i] = 5 * np.log10(DL) + 25 if DL > 0 else np.inf
    return mu_th_array

print("理論曲線の計算を開始します...")
mu_theory_axis = get_mu_theory_for_curve(z_theory, 1.0, cosmo_params_tuple)
mu_theory_equator = get_mu_theory_for_curve(z_theory, 0.0, cosmo_params_tuple)

# --- Calculation of Theoretical Values for Residual Plot ---
# --- 残差プロット用の理論値計算 ---
l_d = 264.02 * u.deg
b_d = 48.25 * u.deg
d_coord_galactic = SkyCoord(l=l_d, b=b_d, frame='galactic')
d_vector = d_coord_galactic.cartesian.xyz.value
sn_coords_icrs = SkyCoord(ra=pantheon_plot_data['RA'], dec=pantheon_plot_data['DEC'], unit='deg', frame='icrs')
sn_coords_galactic = sn_coords_icrs.galactic
n_vectors_sn = sn_coords_galactic.cartesian.xyz.value.T
pantheon_plot_data['cos_theta'] = np.dot(n_vectors_sn, d_vector)

mu_theory_at_data_points = np.zeros(len(pantheon_plot_data))
print("\n残差プロット用の理論値を計算中...")
for i, row in tqdm(pantheon_plot_data.iterrows(), total=len(pantheon_plot_data)):
    integrand = lambda z_prime: 1.0 / hubble_parameter_interaction(z_prime, row['cos_theta'], *cosmo_params_tuple)
    chi, _ = quad(integrand, 0, row['zCMB'])
    DL = c_kms * (1 + row['zCMB']) * chi
    mu_th_model_only = 5 * np.log10(DL) + 25 if DL > 0 else np.inf
    mu_theory_at_data_points[i] = mu_th_model_only + M_fit


# ===== Step 4: Plotting the Graph =====
# ===== ステップ4: グラフの描画 =====
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 12), sharex=True, gridspec_kw={'height_ratios': [3, 1]})
title_params = f'$H_0={best_fit_params["H0"]:.2f}, \\Omega_m={best_fit_params["Omega_m"]:.2f}, A_2={best_fit_params["A2"]:.2f}, \\Gamma_0={best_fit_params["Gamma0"]:.2f}, M={best_fit_params["M"]:.2f}$'
fig.suptitle(f"DIRT相互作用モデルの最適フィットと観測データの比較（最終結果）\n({title_params})", fontsize=16, fontproperties=font_prop)

# Observational Data Points
# 観測データ点
ax1.errorbar(pantheon_plot_data['zCMB'], pantheon_plot_data['m_b_corr'],
             yerr=pantheon_plot_data['m_b_corr_err_DIAG'],
             fmt='o', color='gray', alpha=0.4, label='Pantheon+ データ点', markersize=3, zorder=2)

# Theoretical Curve
# 理論曲線
ax1.plot(z_theory, mu_theory_axis + M_fit, color='red', linestyle='-', label=r'DIRTモデル理論値（軸方向, $\cos\theta=1$）', zorder=3)
ax1.plot(z_theory, mu_theory_equator + M_fit, color='blue', linestyle='--', label=r'DIRTモデル理論値（赤道方向, $\cos\theta=0$）', zorder=3)

ax1.set_ylabel("見かけの距離指数 $\\mu$ (等級)", fontproperties=font_prop)
ax1.legend(prop=font_prop)
ax1.grid(True)
ax1.invert_yaxis()

# Residual Plot
# 残差プロット
residuals = pantheon_plot_data['m_b_corr'] - mu_theory_at_data_points
ax2.errorbar(pantheon_plot_data['zCMB'], residuals,
             yerr=pantheon_plot_data['m_b_corr_err_DIAG'],
             fmt='o', color='black', markersize=3, alpha=0.5)
ax2.axhline(0, color='red', linestyle='--')
ax2.set_xlabel("赤方偏移 z", fontproperties=font_prop)
ax2.set_ylabel("残差 (等級)", fontproperties=font_prop)
ax2.grid(True)
ax2.set_ylim(-2, 2)

plt.tight_layout(rect=[0, 0, 1, 0.94])
plt.show()